In [90]:
# Import libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.feature_selection import mutual_info_classif
from sklearn.feature_selection import chi2
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest
from imblearn.under_sampling import RandomUnderSampler
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import StackingClassifier
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.pipeline import make_pipeline

In [98]:
# Load in data
train_data = pd.read_csv('train_dataset.csv')
test_data = pd.read_csv('test_dataset.csv')

# Remove missing values
train_data.dropna(axis=0, inplace=True)



# Save id column for later Kaggle submission
id_col = test_data['id']
train_data = train_data.drop(['id'], axis=1)
test_data = test_data.drop(['id'], axis=1)

# Check which countries have the most high rated appearances
filtered_df = train_data[train_data['imdb_score_binned'] >= 3]
print(filtered_df[['country', 'imdb_score_binned']])
# Count the occurrences of each country
country_counts = filtered_df['country'].value_counts()

# Sort the counts in descending order
sorted_country_counts = country_counts.sort_values(ascending=False)

# Print the sorted counts
#print(sorted_country_counts)

# Replace top 3 rated countries
def map_country(country):
    if country in high_rated_countries:
        return country
    else:
        return 'RestOfWorld'

high_rated_countries = ['USA', 'UK', 'France']
train_data['country'] = train_data['country'].map(map_country)
test_data['country'] = test_data['country'].map(map_country)

# Drop redundant numeric data
redundant_attributes = ['actor_1_facebook_likes', 'actor_2_facebook_likes', 'actor_3_facebook_likes', 'facenumber_in_poster', 'average_degree_centrality']
train_data = train_data.drop(columns=redundant_attributes, axis=1)
test_data = test_data.drop(columns=redundant_attributes, axis=1)

# Split into attributes and labels
X = train_data.iloc[:, :-1]
y = train_data.iloc[:, -1]

# Drop categorical/nominal data except country, genres and content_rating
numeric_attributes = X.select_dtypes(include='number').columns
numeric_attributes = pd.Index(list(numeric_attributes) + ['country'])
X = X[numeric_attributes]
test_data = test_data[numeric_attributes]

print(X.head())
print(train_data.head())


          country  imdb_score_binned
0             USA                  4
3             USA                  4
4     New Zealand                  4
5             USA                  3
7             USA                  3
...           ...                ...
2997          USA                  3
3000           UK                  4
3001          USA                  3
3002       France                  3
3003       Mexico                  3

[905 rows x 2 columns]
   num_critic_for_reviews  duration  director_facebook_likes      gross  \
0                     186        73                       28  422783777   
1                     252        97                        0   20433940   
2                     232       117                      234     371897   
3                     297       109                        0   13782838   
4                     297       171                        0  313837577   

   num_voted_users  cast_total_facebook_likes  num_user_for_reviews  \
0         

In [94]:
rf = RandomForestClassifier()
rf.fit(X,y)
importances = rf.feature_importances_
print(sorted(zip(importances, X.columns)))
important_features = X.columns[importances >= 0.07]
print(important_features)
#X = X[important_features]


corr_matrix = X.corr().abs()
# Select upper triangle of correlation matrix
upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(np.bool_))

# Find features with correlation greater than a threshold
high_correlation_features = [column for column in upper.columns if any(upper[column] > 0.7)]
print(high_correlation_features)
# Remove one of each highly correlated feature pair
X_filtered = X.drop(high_correlation_features, axis=1)


ValueError: could not convert string to float: 'Roger Allers'

In [76]:
base_estimators = [
    ('gradient_boosting', GradientBoostingClassifier(n_estimators=200, random_state=42)),
    ('random_forest', RandomForestClassifier(n_estimators=200, criterion='entropy', random_state=42)),
    ('logistic_regression', LogisticRegression()),
]
chosen_model = StackingClassifier(estimators=base_estimators,final_estimator=LogisticRegression(), cv=5)

In [77]:
# PIPELINE WITH JUST SCALING THE DATA AND THEN USING RANDOM FOREST
pipeline = make_pipeline(StandardScaler(), chosen_model)
pipeline.fit(X, y)

kfolds = StratifiedKFold(n_splits = 5, shuffle=True, random_state=42)
cv_scores = cross_val_score(pipeline, X, y, cv=kfolds, scoring='accuracy')

print("CV Scores:", cv_scores)
print("Mean CV Score:", cv_scores.mean())


c:\Users\j_awe\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
c:\Users\j_awe\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

CV Scores: [0.6655574  0.65058236 0.65557404 0.64833333 0.66      ]
Mean CV Score: 0.6560094287298947


c:\Users\j_awe\anaconda3\Lib\site-packages\sklearn\linear_model\_logistic.py:458: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
